In [1]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [selenium]/11 [selenium]-client]
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [webdriver-manager]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
from openpyxl import Workbook
import csv
import time

# 크롬 옵션 설정
options = Options()
options.add_argument("--start-maximized")
# options.add_argument("--headless")  # 창 없이 실행하고 싶다면 주석 해제

# 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 유튜브 쇼츠 URL
url = "https://youtube.com/shorts/jKx_ZouWZLs?si=QLAGlu2Iaq0_a6wU"
driver.get(url)
time.sleep(5)

# 제품 정보 저장 리스트
product_data = []

try:
    # "제품 보기" 버튼 클릭
    product_button = driver.find_element(By.XPATH, '//button[contains(text(), "제품 보기")]')
    product_button.click()
    print("✅ 제품 보기 버튼 클릭 완료")
    time.sleep(5)  # 제품 목록 로딩 대기

    # 제품 목록
    products = driver.find_elements(By.CSS_SELECTOR, '[href*="shopping"]')
    print(f"\n🔍 총 {len(products)}개의 제품 감지됨")

    for idx, product in enumerate(products, 1):
        name = "없음"
        price = "없음"
        brand_or_info = "없음"
        img_url = "없음"
        link = product.get_attribute("href")

        # 제품명
        if product.text:
            name = product.text.strip().split("\n")[0]

        # 이미지
        try:
            img = product.find_element(By.TAG_NAME, "img")
            img_url = img.get_attribute("src")
        except:
            pass

        # 추가 정보 (가격, 브랜드/기타)
        try:
            info_elements = product.find_elements(By.XPATH, ".//span")
            for elem in info_elements:
                text = elem.text.strip()
                if "₩" in text or "$" in text:
                    price = text
                elif text and text != name:
                    brand_or_info = text
        except:
            pass

        product_data.append([name, price, brand_or_info, img_url, link])

except Exception as e:
    print("❌ 오류 발생:", e)

driver.quit()

# -----------------------------------------
# 1. CSV 저장
csv_filename = f"shorts_products_{datetime.now():%Y%m%d}.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["제품명", "가격", "브랜드/기타", "이미지 URL", "링크"])
    writer.writerows(product_data)
print(f"✅ CSV 파일 저장 완료: {csv_filename}")

# -----------------------------------------
# 2. Excel 저장
wb = Workbook()
ws = wb.active
ws.title = f"shorts_products_{datetime.now():%Y%m%d}.xlsx"
ws.append(["제품명", "가격", "브랜드/기타", "이미지 URL", "링크"])
for item in product_data:
    ws.append(item)

excel_filename = "shorts_products.xlsx"
wb.save(excel_filename)
print(f"✅ 엑셀 파일 저장 완료: {excel_filename}")


/Users/mac/Desktop/minmin/intern/rhoonart/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


❌ 오류 발생: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[contains(text(), "상품 보기")]"}
  (Session info: chrome=136.0.7103.93); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010c5fe8b8 chromedriver + 5986488
1   chromedriver                        0x000000010c5f59ea chromedriver + 5949930
2   chromedriver                        0x000000010c0ae600 chromedriver + 415232
3   chromedriver                        0x000000010c100304 chromedriver + 750340
4   chromedriver                        0x000000010c100521 chromedriver + 750881
5   chromedriver                        0x000000010c150204 chromedriver + 1077764
6   chromedriver                        0x000000010c12633d chromedriver + 906045
7   chromedriver                        0x000000010c14d566 chromedriver + 1066342
8   chromedriver      